# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd 

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
hygiene_result = establishments.find(query)
pprint(hygiene_result[0])


In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(list(hygiene_result))

# Display the number of rows in the DataFrame
len(hygiene_df)

In [ ]:
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex': 'London'},
         'RatingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
establishments.count_documents(query)

# Display the first document in the results using pprint
london_result = establishments.find(query)
pprint(london_result[0])

In [ ]:
# Convert the result to a Pandas DataFrame
london_df = pd.DataFrame(list(london_result))

# Display the number of rows in the DataFrame
len(london_df)

In [ ]:
# Display the first 10 rows of the DataFrame
london_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =  {'geocode': {'latitude': '51.49014200'}} 
longitude = {'geocode': {'longitude': '0.08384000'}} 


query = {'RatingValue': 5}
sort =  [('scores.Hygiene', -1)]

# Print the results
print(list(establishments.find(query).sort(sort)))
                      

In [ ]:
# Convert result to Pandas DataFrame
location_result = list(establishments.find(query).sort(sort))
location_df = pd.DataFrame(location_result)
location_df.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [{'$match':{'scores.Hygiene':0}},
            {'$group':{'_id':'$LocalAuthorityName','count': {'$sum': 1 }}},
            {'$sort':{'count':-1}}]

pipeline_results = list(establishments.aggregate(pipeline))

# Print the number of documents in the results
print(len(pipeline_results))

# Print the first 10 results
pprint(pipeline_results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.DataFrame(pipeline_results)

# Display the number of rows in the DataFrame
len(pipeline_df)

# Display the first 10 rows of the DataFrame
pipeline_df.head(10)